# First Task: Table

Install and import libraries

In [1]:
!pip install bs4
#!pip install requests

     |████████████████████████████████| 122kB 13.9MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4


In [23]:
from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page
import pandas as pd

Define URL and load it into beauifulsoup

In [76]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [77]:
data  = requests.get(url).text 

In [78]:
soup = BeautifulSoup(data,"html5lib")  # create a soup object using the variable 'data'

In [79]:

#for linebreak in soup.find_all('br'):
 #   linebreak.extract()
    
#find a html table in the web page
table = soup.find('table') # in html table is represented by the tag <table>


Print to have the tree

In [80]:
print(table.prettify())

<table cellpadding="2" cellspacing="0" rules="all" style="width:100%;">
 <tbody>
  <tr>
   <td style="width:11%;">
    <p>
     M1A
     <br/>
     <span style="font-size:85%;">
      Not assigned
     </span>
    </p>
   </td>
   <td style="width:11%;">
    <p>
     M2A
     <br/>
     <span style="font-size:85%;">
      Not assigned
     </span>
    </p>
   </td>
   <td style="width:11%;">
    <p>
     M3A
     <br/>
     <span style="font-size:85%;">
      <a href="/wiki/North_York" title="North York">
       North York
      </a>
      <br/>
      (
      <a href="/wiki/Parkwoods" title="Parkwoods">
       Parkwoods
      </a>
      )
     </span>
    </p>
   </td>
   <td style="width:11%;">
    <p>
     M4A
     <br/>
     <span style="font-size:85%;">
      <a href="/wiki/North_York" title="North York">
       North York
      </a>
      <br/>
      (
      <a href="/wiki/Victoria_Village" title="Victoria Village">
       Victoria Village
      </a>
      )
     </span>
    </p>


Parse through the tree, using the rules defined in the task

In [110]:
data = pd.DataFrame(columns=["PostalCode", "Borough", "Neighborhood"])

for row in table.tbody.find_all("tr"):                                    # parsing all trs
    for col in row.find_all("td"):                                        # also all tds
        if (col != []):
            for p in col.find_all("p"):                                   # the ps contain the data of the cells
                for span in p.find_all("span"):                           # then spans contain the rest 
                    if span != []:
                        if span.text.split('(')[0] != 'Not assigned':
                            code = p.find("br").previousSibling                              # the br separates the code with the rest
                            bor = span.text.split('(')[0]                                    # ( separates the borough with the neighborhood)
                            nei = span.text.split('(')[1].replace(' /', ',').replace(')','') # we want , as separator, ) has to be removed
                            if span.text.split('(')[1].replace(' /', ',').replace(')','') == 'Not assigned':  # if not assigned, use borough
                                nei = bor
                            data = data.append({"PostalCode":code, "Borough":bor, "Neighborhood":nei}, ignore_index=True)
                                    
data

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


The shape as defined in the task:

In [108]:
data.shape

(103, 3)

# Second Task: Locations

Read data

In [119]:
positions = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv')
positions

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


using the data

In [147]:
lat = []
lon = []
for index, row in data.iterrows():
    fit_row = positions[positions['Postal Code'] == row['PostalCode']]
    #print(fit_row.values)
    lat.append(fit_row.values[0][1])
    lon.append(fit_row.values[0][2])

data_2 =data.copy()
data_2['Latitude'] = lat
data_2['Longitude'] = lon
data_2


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


# Third Task: Analysis and Visualization

In [148]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [154]:
# create map of Manhattan using latitude and longitude values
map_toronto = folium.Map(location=[43.651070, -79.347015], zoom_start=10.5)

# add markers to map
for lat, lng, label in zip(data_2['Latitude'], data_2['Longitude'], data_2['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto